In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import requests
from collections import Counter

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [3]:
home_url = 'http://millercenter.org/president/speeches'

In [4]:
page = requests.get(home_url)

In [5]:
soup = BeautifulSoup(page.text, "lxml")

In [6]:
links = soup.find_all("a")

In [7]:
links = soup.findAll(attrs={'class':'field-content'})

In [8]:
for link in links:
    if "icon" not in link.text:
        print(link.text)

February 28, 2017: Address to Joint Session of Congress
January 20, 2017: Inaugural Address
May 15, 2016: Commencement Address at Rutgers University
March 22, 2016: Remarks to the People of Cuba
January 12, 2016: 2016 State of the Union Address
June 26, 2015: Remarks in Eulogy for the Honorable Reverend Clementa Pickney
March 7, 2015: Remarks at the 50th Anniversary of the Selma Marches
January 20, 2015: 2015 State of the Union Address
November 20, 2014: Address to the Nation on Immigration
January 28, 2014: 2014 State of the Union Address
December 4, 2013: Speech on Economic Mobility
September 10, 2013: Address to the Nation on Syria


In [9]:
driver.get(home_url)

x = driver.find_elements_by_tag_name('a')
speeches = list()
for link in x:
    if not pd.isnull(link.get_attribute("href")) and "/presidential-speeches/" in link.get_attribute("href"):
        speeches.append(link.get_attribute("href"))

In [10]:
speech = speeches[0]
response = requests.get(speech)
print(speech)
a = BeautifulSoup(response.text, "lxml")
b = a.findAll(attrs={'class':'transcript-inner'})
len(b)

https://millercenter.org/the-presidency/presidential-speeches/february-28-2017-address-joint-session-congress


1

In [62]:
obama_url = "https://millercenter.org/president/obama"
obama_response = requests.get(obama_url)
obama_soup = BeautifulSoup(obama_response.text, "lxml")
facts = obama_soup.find_all(attrs={'class':'fast-facts-wrapper'})
name = obama_soup.find(attrs={'class': "fullname-data"})
name = name.text.split("\n")[2]
dates = obama_soup.find_all(attrs={"class": "date-data"})
dates = [(i.text.strip()) for i in dates]
info = obama_soup.find_all(attrs={"class": "string-data"})
info = [(i.text.strip()) for i in info]
number = obama_soup.find(attrs={"class": "number-data"})
number = int(number.text.split("\n")[3])

In [37]:
potus_url = "https://millercenter.org/president"
potus_response = requests.get(potus_url)
potus_soup = BeautifulSoup(potus_response.text, "lxml")
presidents = list()
x = potus_soup.find_all("a")
for link in x:
    if "href" in link.attrs:
        href = link.attrs["href"]
        if "/president/" in href:
            presidents.append(href)
presidents = list(set(presidents))

In [38]:
presidents

['/president/vanburen',
 '/president/arthur',
 '/president/washington',
 '/president/adams',
 '/president/taft',
 '/president/lbjohnson',
 '/president/trump',
 '/president/ford',
 '/president/clinton',
 '/president/fdroosevelt',
 '/president/lincoln',
 '/president/garfield',
 '/president/hayes',
 '/president/hoover',
 '/president/mckinley',
 '/president/coolidge',
 '/president/pierce',
 '/president/nixon',
 '/president/bharrison',
 '/president/bush',
 '/president/grant',
 '/president/taylor',
 '/president/fillmore',
 '/president/tyler',
 '/president/roosevelt',
 '/president/jefferson',
 '/president/truman',
 '/president/kennedy',
 '/president/wilson',
 '/president/gwbush',
 '/president/eisenhower',
 '/president/jackson',
 '/president/buchanan',
 '/president/monroe',
 '/president/cleveland',
 '/president/polk',
 '/president/reagan',
 '/president/obama',
 '/president/johnson',
 '/president/madison',
 '/president/carter',
 '/president/jqadams',
 '/president/harrison',
 '/president/harding

In [102]:
platform_url = "http://www.presidency.ucsb.edu/ws/index.php?pid=25849"
platform_response = requests.get(platform_url)
platform_soup = BeautifulSoup(platform_response.text, "lxml")
title = platform_soup.find(attrs={"class": "paperstitle"}).text
headlines = platform_soup.find_all("h3")
topics = list()
for a in headlines:
    topics.append(a.text)
topics = ", ".join(topics)
platform = list()
paras = platform_soup.find_all("p")
for p in paras:
    platform.append(p.text)
platform = " ".join(platform)
year = int(title.split(" ")[0])
party = title.split(" ")[1:3]
party = " ".join(party)
row = [{"Year": year, "Party": party, "Topics": topics, "Platform":platform}]
temp_df = pd.DataFrame(row)


In [107]:
temp_df

,Party,Platform,Topics,Year
0,Republican Party,We meet at a remarkable time in the life of ou...,"Preamble, Old Truths For The New Economy, The ...",2000


In [126]:
urls = ["http://www.presidency.ucsb.edu/ws/index.php?pid=25849", "http://www.presidency.ucsb.edu/ws/index.php?pid=29612"]
platforms = "http://www.presidency.ucsb.edu/platforms.php"
response = requests.get(platforms)
soup = BeautifulSoup(response.text, "lxml")
arr = soup.find_all("a")
links = []
for div in arr:
    if "href" in div.attrs:
        if "?pid=" in div.attrs["href"]:
            links.append(div.attrs["href"])


In [160]:
platform_first = list()
platform_last = list()
for url in links:
    platform_response = requests.get(url)
    platform_soup = BeautifulSoup(platform_response.text, "lxml")
    title = platform_soup.find(attrs={"class": "paperstitle"}).text
    if title.split(" ")[-1] == "Platform":
        platform_last.append(url)
    elif title.split(" ")[-1] != "Party":
        platform_first.append(url)
platform_first

['http://www.presidency.ucsb.edu/ws/index.php?pid=25848',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25847',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25846',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25845',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25844',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25843',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=29615',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25842',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=29570',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25841',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25840',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25839',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25838',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25837',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25836',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25835',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=29640',
 'http://www.p

In [164]:
rows = list()

In [165]:
for url in platform_last:
    platform_response = requests.get(url)
    platform_soup = BeautifulSoup(platform_response.text, "lxml")
    title = platform_soup.find(attrs={"class": "paperstitle"}).text
    print(title)
    headlines = platform_soup.find_all("h3")
    topics = list()
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("h4")
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("b")
    for a in headlines:
        if "1. " not in a.text and "2. " not in a.text:
            topics.append(a.text)
    topics = ", ".join(topics)
    platform = list()
    paras = platform_soup.find_all("p")
    for p in paras:
        platform.append(p.text)
    platform = " ".join(platform)
    year = int(title.split(" ")[0])
    party = title.split(" ")[1]
    #row = [{"Year": year, "Party": party, "Topics": topics, "Platform":platform}]
    row = [year, party, topics, platform]
    rows.append(row)

2016 Democratic Party Platform
2016 Republican Party Platform
2012 Democratic Party Platform
2012 Republican Party Platform
2008 Democratic Party Platform
2008 Republican Party Platform
2004 Democratic Party Platform
2004 Republican Party Platform
2000 Democratic Party Platform
2000 Republican Party Platform
1996 Democratic Party Platform
1992 Democratic Party Platform
1988 Democratic Party Platform
1984 Democratic Party Platform
1980 Democratic Party Platform
1976 Democratic Party Platform
1972 Democratic Party Platform
1968 Democratic Party Platform
1964 Democratic Party Platform
1960 Democratic Party Platform
1956 Democratic Party Platform
1952 Democratic Party Platform
1948 Democratic Party Platform
1944 Democratic Party Platform
1940 Democratic Party Platform
1936 Democratic Party Platform
1932 Democratic Party Platform
1928 Democratic Party Platform
1924 Democratic Party Platform
1920 Democratic Party Platform
1916 Democratic Party Platform
1912 Democratic Party Platform
1908 Dem

In [166]:
for url in platform_first:
    platform_response = requests.get(url)
    platform_soup = BeautifulSoup(platform_response.text, "lxml")
    title = platform_soup.find(attrs={"class": "paperstitle"}).text
    print(title)
    headlines = platform_soup.find_all("h3")
    topics = list()
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("h4")
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("b")
    for a in headlines:
        if "1. " not in a.text and "2. " not in a.text:
            topics.append(a.text)
    topics = ", ".join(topics)
    platform = list()
    paras = platform_soup.find_all("p")
    for p in paras:
        platform.append(p.text)
    platform = " ".join(platform)
    year = int(title.split(" ")[-1])
    party = title.split(" ")[0]
    #row = [{"Year": year, "Party": party, "Topics": topics, "Platform":platform}]
    row = [year, party, topics, platform]
    rows.append(row)

Republican Party Platform of 1996
Republican Party Platform of 1992
Republican Party Platform of 1988
Republican Party Platform of 1984
Republican Party Platform of 1980
Republican Party Platform of 1976
Libertarian Party Platform of 1972
Republican Party Platform of 1972
American Independent Party Platform of 1968
Republican Party Platform of 1968
Republican Party Platform of 1964
Republican Party Platform of 1960
Republican Party Platform of 1956
Republican Party Platform of 1952
Republican Party Platform of 1948
Republican Party Platform of 1944
Republican Party Platform of 1940
Republican Party Platform of 1936
Republican Party Platform of 1932
Republican Party Platform of 1928
Progressive Party Platform of 1924
Republican Party Platform of 1924
Republican Party Platform of 1920
Republican Party Platform of 1916
Progressive Party Platform of 1912
Republican Party Platform of 1912
Republican Party Platform of 1908
Republican Party Platform of 1904
Republican Party Platform of 1900
R

In [169]:
platform_df = pd.DataFrame(rows, columns=["Year", "Party", "Topics", "Platform"])

In [172]:
platform_df = platform_df.sort_values("Year", ascending=False).reset_index()

In [228]:
url = "http://www.presidency.ucsb.edu/nomination.php"
driver.get(url)
a = driver.find_elements_by_xpath("//tr")
for item in a[37:]:
    print(item.text)


• 2001 Transition
• White House Media Pool Reports
Data Archive
Data Index
Media Archive
Audio/Video Index
Elections
Election Index
Florida 2000
Links
Presidential Libraries
• Interesting Convention Facts
• Ballots Required to Nominate Candidates at National Conventions
• Cities That Have Hosted National Conventions
• Political Party Platforms
• Nomination Acceptance Speeches
• Days Before or After the First Day of Convention Vice-Presidential Nominees Were Selected

Presidential Nomination
Acceptance Speeches and Letters
1880 - 2016

compiled by Gerhard Peters

     Democratic Party
Nominee Date Location Length
did nominee win state hosting convention?
in time in words
Hillary Clinton July 28, 2016 Philadelphia, PA 56:34 5389 TBD
Barack Obama September 6, 2012 Charlotte, NC 39:03 4474  no
Barack Obama August 28, 2008 Denver, CO 41:40 4652  yes
John F. Kerry July 29, 2004 Boston, MA 45:26 5166  yes
Albert Gore, Jr. August 17, 2000 Los Angeles, CA 50:20 5755  yes
William J. Clinton Augu

In [3]:
url = "http://www.presidency.ucsb.edu/platforms.php"
driver.get(url)
a = driver.find_elements_by_xpath("//tr")
year = 2016
rows = list()
for item in a[55:199]:
    
    if len(item.text.split(" ")) > 1:
        arr = item.text.split(" ")
        
        arr = list(filter(None, arr))
        #print(arr)
        if "--" in arr:
            arr.remove("--")
        try:
            year = int(arr[0])
            arr.remove(arr[0])
        except:
            year=year
        if "pdf" in arr:
            arr.remove("pdf")
        word_count = arr[-1]
        arr = arr[:-1]
        if arr[0] == "States" or arr[0] == "Constitutional":
            party=" ".join(arr[0:2])
            arr = arr[2:]
        else:
            party = arr[0]
            arr = arr[1:]
        try:
            electoral_votes = int(arr[-1])
            arr = arr[:-1]
        except:
            electoral_votes = 0
        name = " ".join(arr)
        final = [name, year, party, electoral_votes]
        rows.append(final)
nominees_df = pd.DataFrame(rows, columns=["Name", "Year", "Party", "Electoral Votes"])

In [255]:
a = ['2016', 'Democratic', 'pdf', 'Hillary', 'Clinton', '26,058']
b = ['Republican', 'pdf', 'Donald', 'Trump', '35,467']

In [258]:
try:
    x = int(b[0])
except:
    x = 5

In [279]:
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/nominees.pkl", "wb") as picklefile:
    pickle.dump(nominees_df, picklefile)

In [293]:
hrc_url = "http://www.presidency.ucsb.edu/ws/index.php?pid=118051"
hrc_response = requests.get(hrc_url)
hrc_soup = BeautifulSoup(hrc_response.text, "lxml")
speech_name = hrc_soup.find(attrs={"class": "paperstitle"}).text
year = hrc_soup.find(attrs={"class": "docdate"}).text.split(", ")[1]
speech = list()
speech_parts = hrc_soup.find_all("p")
for div in speech_parts:
    speech.append(div.text)
speech = " ".join(speech)

In [315]:
noms_url = "http://www.presidency.ucsb.edu/nomination.php"
noms_response = requests.get(noms_url)
noms_soup = BeautifulSoup(noms_response.text, "lxml")
links = noms_soup.find_all("a")
nom_links = list()
for link in links:
    href = link.attrs["href"]
    if "pid=" in href:
        nom_links.append(href)
len(nom_links)
    

55

In [340]:
type(nom_links)

list

In [348]:
arr = list()
rows = list()
for index, link in enumerate(nom_links):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, "lxml")
    speech_name = soup.find(attrs={"class": "paperstitle"}).text
    if index<24:
        party = "Democratic"
    else:
        party = "Republican"
    year = soup.find(attrs={"class": "docdate"}).text.split(", ")[1]
    speech = list()
    speech_parts = soup.find_all("p")
    for div in speech_parts:
        speech.append(div.text)
    speech = " ".join(speech)
    row = [year, party, speech_name, speech]
    rows.append(row)
#     if "Democratic" not in speech_name and "Republican" not in speech_name:
#         print(link)

        

In [349]:
convention_df = pd.DataFrame(rows, columns=["Year", "Party", "Speech Name", "Speech"])

Counter(arr)

In [350]:
convention_df

,Year,Party,Speech Name,Speech
0,2016,Democratic,Address Accepting the Presidential Nomination ...,Thank you all for the great convention that we...
1,2012,Democratic,693 - Remarks Accepting the Presidential Nomin...,The President. Thank you. Thank you. Thank you...
2,2008,Democratic,Address Accepting the Presidential Nomination ...,"With profound gratitude and great humility, I ..."
3,2004,Democratic,Address Accepting the Presidential Nomination ...,We are here tonight because we love our countr...
4,2000,Democratic,Address Accepting the Presidential Nomination ...,Thank you. Thank you very much. Thank you. Tha...
5,1996,Democratic,Remarks Accepting the Presidential Nomination ...,So many have contributed to the record we have...
6,1992,Democratic,Address Accepting the Presidential Nomination ...,He said he came here tonight because he always...
7,1988,Democratic,Address Accepting the Presidential Nomination ...,A few months ago when Olympia Dukakis in front...
8,1984,Democratic,Address Accepting the Presidential Nomination ...,"My fellow Democrats, my fellow Americans: I ac..."
9,1980,Democratic,Remarks Accepting the Presidential Nomination ...,I thank you for the nomination you've offered ...


In [351]:
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/convention_speeches.pkl", "wb") as picklefile:
    pickle.dump(convention_df, picklefile)

In [7]:
nominees_df.at[0, "Electoral Votes"] = 227
nominees_df.at[1, "Electoral Votes"] = 304
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/nominees.pkl", "wb") as picklefile:
    pickle.dump(nominees_df, picklefile)